# checklist
Author: inryeol
Date: 2024-08-04

In [1]:
# 초기 설정
import pandas as pd
import numpy as np

In [ ]:
# 데이터를 처음부터 다시 설정
# equivalent to rm(list=ls()) in R
%reset -f

In [ ]:
# Options
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.2f}'.format

# **예시 데이터와 질문**

미국 환경부에서 따온 데이터를 가지고 질문 만들기

-   어느 곳의 오염도가 더 심각한가? 동부 vs 서부

좀 더 심도 있게 질문을 만든다면 다음과 같다.

-   시간당 오존 레벨이 평균적으로 높은 곳은? 뉴욕 vs LA
-   지역 ⇒ 대표성을 띄는 도시로 축소

## 파일 불러오기

In [ ]:
ozone = pd.read_csv('../../dataset/hourly_44201_2014_ver2.csv')
ozone.columns = ozone.columns.str.replace(' ', '_')
ozone.head()

## 초기분석

In [ ]:
ozone.shape

In [ ]:
ozone.info()

In [ ]:
ozone.head()

In [ ]:
ozone.tail()

## 결측값 보기

-   샘플수에 대해서 민감해야 한다.
-   샘플수가 30개 정도일때 missing이 있으면 값이 확확 줄어들 수 있다.

미국의 hourly ozone data를 보자.

In [ ]:
ozone['Time_Local'].value_counts()

미국의 주를 기준으로 분석한다고 할 때, 주가 아닌 것들이 있다. 주의 이름을 나열해 이들을 찾아보자.

In [ ]:
ozone['State_Name'].unique()

## 이상치 확인

데이터의 이상치 확인해보기.

In [ ]:
ozone['Sample_Measurement'].describe()

In [ ]:
ozone['Sample_Measurement'].quantile(np.arange(0, 1.1, 0.1))

## 조건 걸고 확인

**오존 레벨 top 10인 지역들을 확인해보자.**

In [ ]:
# aggregate 함수로 만들어주기
ranking = ozone.groupby(['State_Name', 'County_Name'], as_index=False)['Sample_Measurement'].mean()
ranking.columns = ['State_Name', 'County_Name', 'ozone']
ranking = ranking.sort_values(by='ozone', ascending=False)
ranking.head(10)

**How many observations in California, Mariposa?**

In [ ]:
mariposa_data = ozone[(ozone['State_Name'] == 'California') & 
                     (ozone['County_Name'] == 'Mariposa')]
mariposa_data.shape[0]

**시간 순으로 체크해보자.**

In [ ]:
mariposa_data.pivot_table(index='Method_Name', columns='Time_Local', aggfunc='size', fill_value=0)

**월별 오존농도를 체크해보자**

In [ ]:
ozone['Date_Local'] = pd.to_datetime(ozone['Date_Local'])
ozone['month'] = ozone['Date_Local'].dt.month_name()
monthly_data = ozone.groupby('month', as_index=False)['Sample_Measurement'].mean()
monthly_data.columns = ['month', 'ozone']
monthly_data

## Challenge Your Solution

부트스트랩 샘플을 이용해 샘플을 뽑아나고, 새 ranking을 만들어보자.

In [ ]:
np.random.seed(10234)
N = ozone.shape[0]
idx = np.random.choice(N, N, replace=True)
ozone2 = ozone.iloc[idx, :]

이후 기존의 ranking과 새 ranking(ranking2)를 붙여보자.

In [ ]:
# ranking2 만들기
ranking2 = ozone2.groupby(['State_Name', 'County_Name'], as_index=False)['Sample_Measurement'].mean()
ranking2.columns = ['State_Name', 'County_Name', 'ozone']
ranking2 = ranking2.sort_values(by='ozone', ascending=False)

# ranking, ranking2 붙이고 확인
pd.concat([ranking.head(10), ranking2.head(10)], axis=1)

In [ ]:
pd.concat([ranking.tail(10), ranking2.tail(10)], axis=1)